In [1]:
import folium
import pandas as pd
import numpy as np
from su

global_path = '/home/louis/summer_invitational_data_open_2020/data/London_in_depth/'
output_map_name = global_path + "london_tubes.html"

# Load data

In [2]:
# Files containing the position of the London underground stations and lines
filename_stations = global_path + 'london_stations.csv'
filename_lines = global_path + 'london_tube_lines.csv'

# Initialize the map

In [3]:
city_of_london_coordinates = [51.51279, -0.09184]
m = folium.Map(location=city_of_london_coordinates, zoom_start=12)
m

# Add the stations

In [4]:
def pipeline_stations(filename_stations: str, max_zone: int = None, current_map: folium.folium.Map = None) -> Dict[(float, float)]:
    # Load stations data
    df = pd.read_csv(filename_stations)
    df['Zone'].fillna('8', inplace=True)
    
    # Map station with coordinates
    unique_stations = df['Station'].unique()
    df.set_index('Station', inplace=True)
    map_station_to_coordinates = {station: (df.loc[station]['Latitude'], 
                                            df.loc[station]['Longitude']) for station in unique_stations}
    
    # Plot the station on the provided map
    if current_map is not None:
        if max_zone is None:
            for station in unique_stations:
                folium.Marker(map_station_to_coordinates[station], popup=station, 
                              icon=folium.Icon(icon='subway', prefix='fa')).add_to(current_map)
        else:
            mask_stations = [station for station in unique_stations 
                                         if int(df.loc[station]['Zone'].split(',')[0]) <= max_zone]
            for station in mask_stations:
                folium.Marker(map_station_to_coordinates[station], popup=station, 
                              icon=folium.Icon(icon='subway', prefix='fa')).add_to(current_map)
        
    return map_station_to_coordinates

In [5]:
dic_stations = pipeline_stations(filename_stations, max_zone=2, current_map=m)
m

# Add the lines

In [6]:
def pipeline_lines(filename_lines: str, filename_stations: str, current_map: folium.folium.Map = None):
    # Load stations data
    df = pd.read_csv(filename_lines)
    dic_stations = pipeline_stations(filename_stations)

    # Get stations for each line
    unique_lines = df['Tube Line'].unique()
    df.set_index('Tube Line', inplace=True)

    # We create a feature group (layer) for each set line
    dic = {line: folium.FeatureGroup(name=line) for line in unique_lines}

    # Add trajectories
    for line in unique_lines:
        infos_line = df.loc[line]
        if len(infos_line) > 2:
            for index, row in infos_line.iterrows():
                # Find start and end stations
                start_station = row['From Station']
                end_station = row['To Station']

                # Add to the correct layer
                folium.PolyLine((dic_stations[start_station], dic_stations[end_station])).add_to(dic[line])
                
    # Add to current map
    for line in dic:
        dic[line].add_to(m)

In [7]:
# city_of_london_coordinates = [51.51279, -0.09184]
# m = folium.Map(location=city_of_london_coordinates, zoom_start=12)
pipeline_lines(filename_lines, filename_stations, m)
m